In [1]:
%matplotlib inline
import pandas as pd

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
sales1 = pd.read_csv('sales1.csv')
sales1

,Book title,Number sold,Sales price,Royalty paid
0,The Bricklayer’s Bible,8,2.99,0.55
1,Swimrand,2,1.99,0.35
2,Pining For The Fisheries of Yore,28,2.99,0.55
3,The Duck Goes Here,34,2.99,0.55
4,The Tower Commission Report,4,11.50,4.25


In [4]:
sales2 = pd.read_csv('sales2.csv')
sales2.fillna('')

,Title,Units sold,List price,Royalty
0,,,,
1,Sales report for Q4,,,
2,E-Book Reader US Store,,,
3,Pining for the Fisheries of Yore,80,3.5,14.98
4,Swimrand,1,2.99,0.14
5,The Bricklayer's Bible,17,3.5,5.15
6,The Duck Goes Here,34,2.99,5.78
7,The Tower Commission Report,4,9.5,6.2
8,US royalties (USD),,,32.25
9,,,,


### Challenge: first combine these sales together into a single dataframe, then compute how much money consumers spent on each book in each currency.

In [100]:
s1 = sales1.copy()
s1['currency'] = 'USD'
s1 = s1.rename(columns={
    'Book title': 'title',
    'Number sold': 'units',
    'Sales price': 'price',
    'Royalty paid': 'royalty'
})
s1

,title,units,price,royalty,currency
0,The Bricklayer’s Bible,8,2.99,0.55,USD
1,Swimrand,2,1.99,0.35,USD
2,Pining For The Fisheries of Yore,28,2.99,0.55,USD
3,The Duck Goes Here,34,2.99,0.55,USD
4,The Tower Commission Report,4,11.50,4.25,USD


In [106]:
s2 = sales2.copy()
s2 = s2.rename(columns={
    'Title': 'title',
    'Units sold': 'units',
    'List price': 'price',
    'Royalty': 'royalty'
})
s2['currency'] = s2.title.str.extract(r'\((.*)\)').fillna(method='bfill')
s2 = s2[s2.units.notnull()]
s2

,title,units,price,royalty,currency
3,Pining for the Fisheries of Yore,80.0,3.50,14.98,USD
4,Swimrand,1.0,2.99,0.14,USD
5,The Bricklayer's Bible,17.0,3.50,5.15,USD
6,The Duck Goes Here,34.0,2.99,5.78,USD
7,The Tower Commission Report,4.0,9.50,6.20,USD
13,Pining for the Fisheries of Yore,47.0,2.99,11.98,GBP
14,The Bricklayer's Bible,17.0,2.99,3.50,GBP
15,The Tower Commission Report,4.0,6.50,4.80,GBP
21,Swimrand,8.0,1.99,0.88,EUR
22,The Duck Goes Here,12.0,1.99,1.50,EUR


In [117]:
s12.groupby(['title', 'currency']).size()

title                             currency
Pining For The Fisheries of Yore  USD         1
Pining for the Fisheries of Yore  GBP         1
                                  USD         1
Swimrand                          EUR         1
                                  USD         2
The Bricklayer's Bible            GBP         1
                                  USD         1
The Bricklayer’s Bible            USD         1
The Duck Goes Here                EUR         1
                                  USD         2
The Tower Commission Report       GBP         1
                                  USD         2
dtype: int64

In [122]:
s12 = s1.append(s2)
s12['total'] = s12.units * s12.price
s12['title'] = s12.title.str.replace('’', "'").str.lower()
s12 = s12[['title', 'currency', 'total']]
s12

,title,currency,total
0,the bricklayer's bible,USD,23.92
1,swimrand,USD,3.98
2,pining for the fisheries of yore,USD,83.72
3,the duck goes here,USD,101.66
4,the tower commission report,USD,46.00
3,pining for the fisheries of yore,USD,280.00
4,swimrand,USD,2.99
5,the bricklayer's bible,USD,59.50
6,the duck goes here,USD,101.66
7,the tower commission report,USD,38.00


In [125]:
s12 = s12.groupby(['title', 'currency']).sum()
s12

total
title                            currency        
pining for the fisheries of yore GBP       140.53
                                 USD       363.72
swimrand                         EUR        15.92
                                 USD         6.97
the bricklayer's bible           GBP        50.83
                                 USD        83.42
the duck goes here               EUR        23.88
                                 USD       203.32
the tower commission report      GBP        26.00
                                 USD        84.00

In [129]:
s12.unstack('currency').fillna(0)

total                
currency                            EUR     GBP     USD
title                                                  
pining for the fisheries of yore   0.00  140.53  363.72
swimrand                          15.92    0.00    6.97
the bricklayer's bible             0.00   50.83   83.42
the duck goes here                23.88    0.00  203.32
the tower commission report        0.00   26.00   84.00